<h1> <center> Assignemnt 1 </center>
 <center> Big Data Systems and Intelligence Analystics </center> </h1> 

# PART - B   --  Analyze Some Data (60 points)

In this part B section, two ML algorithms will be selected to evaluate the performace of the predicting model. 
Based on exploratory data analysis this Bank's Marketing dataset needs to get classify whether their customers are interested in Term Deposit or not. Since the prediction is based on the classification, ML classification algorithms would be a good fit for this case. <br><br>
For this dataset XGBoost and Support Vector Machine are choosen for building the predictive model, which are prefered much on classification type.

### Importing Libraries

In [262]:
import pandas as pd
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (12, 7)
import seaborn as sns
from termcolor import colored
from IPython.core import display as ICD
import warnings
warnings.filterwarnings('ignore')

import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from sklearn import cross_validation, metrics   #Additional scklearn functions
from sklearn.grid_search import GridSearchCV   #Perforing grid search

from sklearn.svm import SVC  
from sklearn.metrics import classification_report, confusion_matrix  

### Reading the data from the output of the Part A analysis

In [32]:
bank_rawdata = pd.read_csv('parta', sep = '\t', header = None)
bank_rawdata.columns = bank_rawdata.iloc[0]
bank_data = bank_rawdata.reindex(bank_rawdata.index.drop(0))
bank_data.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
1,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,0
2,57,services,married,high.school,no,no,no,telephone,may,mon,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,0
3,37,services,married,high.school,no,yes,no,telephone,may,mon,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,0
4,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,0
5,56,services,married,high.school,no,no,yes,telephone,may,mon,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,0


### Converting the Format from numeric object to int64 and float64

In [33]:
bank_data = bank_data.convert_objects(convert_numeric=True)
bank_data.dtypes

0
age                 int64
job                object
marital            object
education          object
default            object
housing            object
loan               object
contact            object
month              object
day_of_week        object
previous            int64
poutcome           object
emp.var.rate      float64
cons.price.idx    float64
cons.conf.idx     float64
euribor3m         float64
nr.employed       float64
y                   int64
dtype: object

### Converting all the categorial values to number through 0 N 1 coding

In [34]:
bank_data = pd.get_dummies(bank_data, columns=["marital", "job", "education", "default", "housing", "loan", "contact",
                                                 "month", "day_of_week", "poutcome"])
bank_data['y'] = bank_data['y'].replace('yes',1)
bank_data['y'] = bank_data['y'].replace('no',0)
bank_data.head()

,age,previous,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y,marital_divorced,marital_married,...,month_oct,month_sep,day_of_week_fri,day_of_week_mon,day_of_week_thu,day_of_week_tue,day_of_week_wed,poutcome_failure,poutcome_nonexistent,poutcome_success
1,56,0,1.1,93.994,-36.4,4.857,5191.0,0,0,1,...,0,0,0,1,0,0,0,0,1,0
2,57,0,1.1,93.994,-36.4,4.857,5191.0,0,0,1,...,0,0,0,1,0,0,0,0,1,0
3,37,0,1.1,93.994,-36.4,4.857,5191.0,0,0,1,...,0,0,0,1,0,0,0,0,1,0
4,40,0,1.1,93.994,-36.4,4.857,5191.0,0,0,1,...,0,0,0,1,0,0,0,0,1,0
5,56,0,1.1,93.994,-36.4,4.857,5191.0,0,0,1,...,0,0,0,1,0,0,0,0,1,0


### Converting the data type of the columns from object to int and float

In [291]:
bank_data = bank_data.convert_objects(convert_numeric=True)
bank_data.dtypes.head()

age                 int64
previous            int64
emp.var.rate      float64
cons.price.idx    float64
cons.conf.idx     float64
dtype: object

************************************************************************************************************************
<br><br><br>

Lets begin with Algothims

# XGBoost Algorithm

### Method to train the model 

In [164]:
train = bank_data
target = 'y'
IDcol = 'ID'
def modelfit(alg, dtrain, predictors,useTrainCV=True, cv_folds=5, early_stopping_rounds=50):
    
    if useTrainCV:
        xgb_param = alg.get_xgb_params()
        xgtrain = xgb.DMatrix(dtrain[predictors].values, label=dtrain[target].values)
        cvresult = xgb.cv(xgb_param, xgtrain, num_boost_round=alg.get_params()['n_estimators'], nfold=cv_folds,
            metrics='auc', early_stopping_rounds=early_stopping_rounds)
        alg.set_params(n_estimators=cvresult.shape[0])
    
    #Fit the algorithm on the data
    alg.fit(dtrain[predictors], dtrain['y'],eval_metric='auc')
        
    #Predict training set:
    dtrain_predictions = alg.predict(dtrain[predictors])
    dtrain_predprob = alg.predict_proba(dtrain[predictors])[:,1]
        
    #Print model report:
    print ("\nModel Report")
    print ("Accuracy : %.4g" % metrics.accuracy_score(dtrain['y'].values, dtrain_predictions))
    print ("AUC Score (Train): %f" % metrics.roc_auc_score(dtrain['y'], dtrain_predprob))


### Evaluating the predicted results on grid search with 2 hyperpameter 

In [176]:
#Choose all predictors except target & IDcols
print("The 2 output metrics are provided below to evaluate the algorithm XGBoost")
print("Based on a grid search on 2 hyperparameter subsample and colsample_bytree")
inputvalue = [0.2,0.4,0.6,0.8,1]
for forloop in inputvalue:
    predictors = [x for x in train.columns if x not in [target, IDcol]]
    xgb1 = XGBClassifier(
     subsample= forloop,
     colsample_bytree= forloop,
     seed=27)
    modelfit(xgb1, train, predictors)

The 2 output metrics are provided below to evaluate the algorithm XGBoost
Based on a grid search on 2 hyperparameter subsample and colsample_bytree

Model Report
Accuracy : 0.9062
AUC Score (Train): 0.796368

Model Report
Accuracy : 0.9076
AUC Score (Train): 0.799991

Model Report
Accuracy : 0.9084
AUC Score (Train): 0.802554

Model Report
Accuracy : 0.9083
AUC Score (Train): 0.803378

Model Report
Accuracy : 0.9086
AUC Score (Train): 0.802983


The Above results shows that when the 2 hyperparameter subsample and colsample_bytree has the maximum value 1 the metrics of the XGBoost Accuracy and AUC increases. The results shows that 90.86% of accuracy and 80.3% of AUC score (Train)<br><br><br>



*************************************************************************************************************************

# Support Vector Machine

The Support Vector Machine algorithm is used as the second model to predict the customers decision on taking the term deposit on bank's marketing data

###### The output dataset of Part A is taken after converting to optimal format for this algorithm

### The dataset is split into input and ouput

In [268]:
bankdata = bank_data
X = bankdata.drop('y', axis=1)  
y = bankdata['y'] 

### Spliting the data into train and test

In [269]:
from sklearn.model_selection import train_test_split  
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20)  

### Defining the 5 set of SVC classifier with different values in hyperparameter

In [287]:
svclassifier1 = SVC(C=1.0, cache_size=200, coef0=0.0,
    degree=3, gamma='auto', kernel='linear',
    max_iter=-1)
svclassifier2 = SVC(C=3.0, cache_size=400, coef0=0.0,
    degree=3, gamma='auto', kernel='linear',
    max_iter=-1)
svclassifier3 = SVC(C=5.0, cache_size=600, coef0=0.0,
    degree=3, gamma='auto', kernel='linear',
    max_iter=-1)
svclassifier4 = SVC(C=7.0, cache_size=800, coef0=0.0,
    degree=3, gamma='auto', kernel='linear',
    max_iter=-1)
svclassifier5 = SVC(C=9.0, cache_size=1000, coef0=0.0,
    degree=3, gamma='auto', kernel='linear',
    max_iter=-1)

### Fitting the model with training dataset 

In [288]:
svclassifier1.fit(X_train, y_train)
svclassifier2.fit(X_train, y_train)
svclassifier3.fit(X_train, y_train)
svclassifier4.fit(X_train, y_train)
svclassifier5.fit(X_train, y_train)

SVC(C=9.0, cache_size=1000, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

### Predicting output for the testing data 

In [289]:
y_pred1 = svclassifier1.predict(X_test) 
y_pred2 = svclassifier2.predict(X_test) 
y_pred3 = svclassifier3.predict(X_test) 
y_pred4 = svclassifier4.predict(X_test) 
y_pred5 = svclassifier5.predict(X_test) 

### Printing the classification report to have different metrics

In [290]:

#print(confusion_matrix(y_test,y_pred))  
print(classification_report(y_test,y_pred1))  
print(classification_report(y_test,y_pred2))
print(classification_report(y_test,y_pred3))
print(classification_report(y_test,y_pred4))
print(classification_report(y_test,y_pred5))

             precision    recall  f1-score   support

          0       0.91      0.98      0.95      7200
          1       0.61      0.20      0.30       866

avg / total       0.88      0.90      0.88      8066

             precision    recall  f1-score   support

          0       0.91      0.98      0.95      7200
          1       0.57      0.22      0.32       866

avg / total       0.88      0.90      0.88      8066

             precision    recall  f1-score   support

          0       0.91      0.98      0.94      7200
          1       0.57      0.18      0.27       866

avg / total       0.87      0.90      0.87      8066

             precision    recall  f1-score   support

          0       0.91      0.98      0.94      7200
          1       0.52      0.20      0.29       866

avg / total       0.87      0.89      0.87      8066

             precision    recall  f1-score   support

          0       0.91      0.98      0.94      7200
          1       0.56      0.22 

The above report shows that support metrics does not have any impact but on the metrics Precision, Recall, F1-score there are slight increase till third report then a small dip in 4th report and again increases in the 5th report. These reports are generated based on the changes on the value of hyperparameter "C" and "cache_size". Has these values increases the output metrics also increase.

### Accuracy of the SVM model

In [301]:
print('Accuracy_1 = ' + str(round(accuracy_score(y_test,y_pred1)*100,2))+'%')
print('Accuracy_2 = ' + str(round(accuracy_score(y_test,y_pred2)*100,2))+'%')
print('Accuracy_3 = ' + str(round(accuracy_score(y_test,y_pred3)*100,2))+'%')
print('Accuracy_4 = ' + str(round(accuracy_score(y_test,y_pred4)*100,2))+'%')
print('Accuracy_5 = ' + str(round(accuracy_score(y_test,y_pred5)*100,2))+'%')


Accuracy_1 = 90.02%
Accuracy_2 = 89.86%
Accuracy_3 = 89.76%
Accuracy_4 = 89.46%
Accuracy_5 = 89.8%


***********************************************************************************************************************

# Conclusion of Part B analysis

    Based on the output of the above two algorithms it is clearly seen that XGBoost predictive model has slightly higher efficiency when compare to the SVM predictive model.
    
    As in all 5 different reports based on the change in value on the hyperparameter, the model is fine-tuned and found the most efficient part of it.
    
    Along with accuracy of each algorithm the perforamce is high is XGBoost when compare to SVM as the SVM model takes more time to build the predictive model.
